In [ ]:
import json, os, requests, ipaddress, folium, math
from collections import defaultdict


FILE_NAME = "/Users/jineiya/Desktop/MET/reports.json"
with open(FILE_NAME, 'r') as f:
   data = json.load(f)

freq = defaultdict(int)
delay_sum = defaultdict(float)
delay_count = defaultdict(int)
# The output of the MTR report containing every IP address crossed to reach the target.


points = {}
# TODO: ajouter les pertes dans le popup
for report_entry in data:
    for timestamp, report_data in report_entry.items():
        seen = set()
        for router in report_data["report"]["hubs"]:
            ip_address = router["host"]


            if ip_address == "???":
                continue

            if ip_address not in seen:
                freq[ip_address] += 1
                seen.add(ip_address)

            avg_delay = router.get("Avg")
            if isinstance(avg_delay, (int, float)):
                delay_sum[ip_address] += avg_delay
                delay_count[ip_address] += 1


       # Get more in-depth info on those IP addresses
for ip in freq:
    url = f"https://ipinfo.io/{ip}"
    r = requests.get(url)
    if r.ok and "loc" in r.json():
        lat, lon = map(float, r.json()["loc"].split(","))
        points[ip] = {
            "lat": lat,
            "long": lon,
            "avg_delay": delay_sum[ip]/delay_count[ip],
            "freq": freq[ip],
        }
   # Handle errors
    else:
        print(f"Error at IP {ip}: {r.status_code}")


first_key = next(iter(points))
# Création de la carte
m = folium.Map(location=[points[first_key]["lat"], points[first_key]["long"]], zoom_start=5)

# Ajout des points
for ip, info in points.items():
    lat, lon = info["lat"], info["long"]
    freq_val = info["freq"]
    delay_val = info["avg_delay"]

    # tracer les cercles
    folium.CircleMarker(
        location=[lat, lon],
        radius=3 + freq_val,
        color="red",
        fill=True,
        fill_color="red",
        fill_opacity=0.6,
        popup=f"{ip}<br>ttr_moyenne: {delay_val:.1f} ms<br>nb_occurrences: {freq_val}",
    ).add_to(m)

    # ajouter les labels
    folium.Marker(
        location=[lat, lon],
        icon=folium.DivIcon(
            html=f"<div style='font-size:9pt; color:black; text-align:center;'>"
                 f"{delay_val:.1f} ms<br>{freq_val}×</div>"
        ),
    ).add_to(m)


m.save("map.html")
print("✅ Carte enregistrée dans map.html")


Error at IP 192.168.1.1: 200
Error at IP 100.127.240.242: 200
Error at IP 172.16.50.33: 200
Error at IP 172.16.50.181: 200
Error at IP 172.16.50.190: 200
Error at IP 172.16.50.198: 200
✅ Carte enregistrée dans map.html
